In [8]:
import pandas as pd 
import psycopg2
import plotly.express as px

# import os

df_saldo=pd.read_excel('G:\Trading\K11\Aluguel\Arquivos\Doar\Saldo-Dia\Kappa_lend_to_day_21-12-2021.xlsx',index_col=0)
# display(df_saldo)
db_conn_risk = psycopg2.connect(host='PGKPTL01', dbname='Risk_DB' , user='kapitalo11', password='kapitalo11')

def get_analysis(ticker):
    query = f"SELECT  corretora, sum(qtde), codigo \
                FROM public.st_alugcustcorr where codigo = '{ticker}' \
                group by  corretora,qtde,codigo order by corretora ASC \
                "
    
    df = pd.read_sql(query,db_conn_risk)
    return df

df_saldo=df_saldo[df_saldo['codigo']=='ITUB4']
for x in df_saldo['codigo'].tolist():
    df =get_analysis(x)
    df['ponta']=df.apply(lambda row: 'T' if row['sum']>0 else 'D',axis=1)
    df=df.groupby(['corretora','ponta'],as_index=False).agg({'sum':sum})
    df_doador=df[df['ponta']=='D']

    df_doador['sum']= df_doador['sum'].apply(lambda x: -x)

    df_doador=df_doador.sort_values(by='sum', ascending=False)
    df_doador=df_doador.reset_index(drop=True)
    df_top_doador=df_doador.iloc[0:10]

    display(df_top_doador)

    fig_a= px.bar(df_top_doador,y="sum",x="corretora",title=x)
    fig_a.show()



# os.system('jupyter nbconvert --execute --to pdf --no-input ../"Analise por corretora.ipynb"')
# df_tomador=df[df['ponta']=='T']



C:\Python37\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,corretora,ponta,sum
0,BRADESCO S/A CTVM,D,1.079781e+09
1,XP INVESTIMENTOS CCTVM S/A,D,8.640307e+08
2,GENIAL INSTITUCIONAL CCTVM S/A,D,5.550903e+08
3,BGC LIQUIDEZ DTVM LTDA,D,5.494870e+08
4,XP INVESTIMENTOS CCTVM S/A,D,4.979901e+08
5,MORGAN STANLEY CTVM S/A,D,4.487280e+08
6,INTL FCStone DTVM Ltda,D,4.341990e+08
7,INTL FCSTONE DTVM LTDA,D,3.628276e+08
8,BTG PACTUAL CTVM S/A,D,3.593187e+08
9,TULLETT PREBON BRASIL CVC LTDA.,D,3.109324e+08
